In [31]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from scipy import signal
from matplotlib import animation, rc
from IPython.display import HTML
import os
from matplotlib.pyplot import figure
%matplotlib inline

In [32]:
def create_video(orig_image,masked_image,shape_properties,cut=0):
    fig, ax = plt.subplots()
    plt.close()
    def animator(N): # N is the animation frame number
        ax.imshow(orig_image[:,:,cut,N], cmap='gray') # I would add interpolation='none'
        data_masked = np.ma.masked_where(masked_image[:,:,cut,N] == 0, masked_image[:,:,cut,N])
        ax.imshow(data_masked,interpolation = 'none', vmin=0, alpha=0.8)
        ax.set_title('W='+str(shape_properties[cut,N,0])+
                     ';H='+str(shape_properties[cut,N,1])+
                     ';A='+str(shape_properties[cut,N,2]))
        return ax
    PlotFrames = range(0,masked_image.shape[3],1)
    anim = animation.FuncAnimation(fig,animator,frames=PlotFrames,interval=100)
    rc('animation', html='jshtml') # embed in the HTML for Google Colab
    return anim

In [33]:
def get_line_eq(p1,p2):
    
    k = (p1[1]-p2[1])/(p1[0]-p2[0])
    c = p1[1] - k*p1[0]
    return (lambda x: k*x+c)

In [34]:
def dist_line_pt(l1,l2,p):
        
    k = (l1[1]-l2[1])/(l1[0]-l2[0])
    n = l1[1] - k*l1[0]
    a,b,c = k,-1,n
    x=p[0]
    y=p[1]
    d = np.abs(a*x + b*y + c)/np.sqrt(a**2+b**2)
    return d

In [35]:
def dist(p1,p2):
    return np.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)

In [43]:
def segment_mitral_valve(image_path, seg_image_path):
    
    nim = nib.load(image_path)
    image = nim.get_fdata()
    seg_nim = nib.load(seg_image_path)
    seg_image = seg_nim.get_fdata()
    pixdim = nim.header['pixdim'][1:4] #* 1e-3
    
    # for loop here
    video_frames = []
    distances = []
    for cut in range(image.shape[2]):
        for frame_id in range(image.shape[3]):
            
            frame = image[:,:,cut,frame_id]
            seg_frame = seg_image[:,:,cut,frame_id]
            
            img =  (frame) * (seg_frame==3)
            img = img/np.max(img)
            imgray = (255*img).astype(np.uint8)
            contours, hierarchy = cv2.findContours(imgray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            mask = np.zeros(img.shape)
            cont = contours[0]
            rect = cv2.minAreaRect(cont)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            savebox=box
            rvline = get_line_eq(box[2],box[3])

            img =  (frame) * (seg_frame==1)
            img = img/np.max(img)
            imgray = (255*img).astype(np.uint8)
            contours, hierarchy = cv2.findContours(imgray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            mask = np.zeros(img.shape)
            cont = contours[0]
            rect = cv2.minAreaRect(cont)
            box = cv2.boxPoints(rect)
            box = np.int0(box)


            a = dist_line_pt(box[1],box[2],savebox[3])
            c = dist(savebox[3], box[2])
            b = np.sqrt(c**2 - a**2)

            lvline = get_line_eq(box[1],box[2])
            X = np.linspace(box[2][0]-b,box[2][0],1000)
            
            video_frames.append([frame,X,lvline(X)])
            distances.append(b*pixdim[0])

    return video_frames,distances

In [46]:
figs,dists = segment_mitral_valve("data/1013344/la_4ch.nii.gz","data/1013344/seg4_la_4ch.nii.gz")

<ipython-input-34-7267d9470b79>:3: RuntimeWarning: divide by zero encountered in long_scalars
  k = (l1[1]-l2[1])/(l1[0]-l2[0])
<ipython-input-34-7267d9470b79>:8: RuntimeWarning: invalid value encountered in double_scalars
  d = np.abs(a*x + b*y + c)/np.sqrt(a**2+b**2)
<ipython-input-33-797257ccda90>:5: RuntimeWarning: invalid value encountered in add
  return (lambda x: k*x+c)
<ipython-input-34-7267d9470b79>:3: RuntimeWarning: invalid value encountered in long_scalars
  k = (l1[1]-l2[1])/(l1[0]-l2[0])


In [47]:
fig, ax = plt.subplots()
plt.close()
def animator(N): # N is the animation frame number
    ax.clear()
    ax.imshow(figs[N][0],cmap='gray')
    ax.plot([figs[N][1][0]],[figs[N][2][0]],'*',alpha=0.5)
    ax.plot([figs[N][1][-1]],[figs[N][2][-1]],'*',alpha=0.5)
    ax.plot(figs[N][1],figs[N][2],'r',alpha=0.8)
    ax.set_title(dists[N])
    return ax
PlotFrames = range(0,len(figs),1)
anim = animation.FuncAnimation(fig,animator,frames=PlotFrames,interval=100)
rc('animation', html='jshtml') # embed in the HTML for Google Colab
anim